In [7]:
import pandas as pd
import pickle
import hashlib

In [8]:
with open(r"C:\Development\ultimateNakMuay\notebooks\pickle_objects\fighter_ufc_payload_async.pkl", "rb") as f:
    results = pickle.load(f)

In [9]:
def cleanResults(result):
    event_name = list(result)[0]
    event_df = result[event_name][0]
    event_df.columns = [_[1] for _ in event_df.columns]
    sub_dfs = []
    start_index = 0

    for i, row in event_df.iterrows():
        if ("Preliminary card" in row["Weight class"]) or (("Preliminary Card" in row["Weight class"])):
            sub_df = event_df[start_index:i]
            sub_dfs.append(sub_df)
            start_index = i + 1

    sub_df = event_df[start_index:]
    sub_dfs.append(sub_df)
    
    for i, sdf in enumerate(sub_dfs):
        if i == 0:
            sdf.loc[:,"event_type"] = "main event"
        else:
            sdf.loc[:,"event_type"] = "preliminary event"
    
    df = pd.concat(sub_dfs)

    cols2rename = {
        x: x.lower().replace(' ', '_') for x in df.columns
    }

    cols2rename = {
        **cols2rename,
        **{
            "Unnamed: 1_level_1": "winner",
            "Unnamed: 3_level_1": "loser",
        },
    }

    df.rename(columns=cols2rename, inplace=True)

    df.drop(columns=["unnamed:_2_level_1"], axis=1, inplace=True)

    df.loc[:, "event_name"] = event_name

    df.loc[:, "event_id"] = df["event_name"].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())

    return df

In [10]:
results = list(map(cleanResults, results))

C:\Users\paulm\AppData\Local\Temp\ipykernel_12876\1973832475.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf.loc[:,"event_type"] = "main event"
C:\Users\paulm\AppData\Local\Temp\ipykernel_12876\1973832475.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf.loc[:,"event_type"] = "preliminary event"
C:\Users\paulm\AppData\Local\Temp\ipykernel_12876\1973832475.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [11]:
final_df = pd.concat(results, ignore_index=True)

In [12]:
final_df

,weight_class,winner,loser,method,round,time,notes,event_type,event_name,event_id
0,Welterweight,Rory MacDonald,Tarec Saffiedine,TKO (punches),3,1:28,NaN,main event,UFC Fight Night: MacDonald vs. Saffiedine,5f12a76ed2214b2a88d6e157c42751360d2a47dd2272dc...
1,Bantamweight,Raphael Assunção,Bryan Caraway,"Decision (unanimous) (30–27, 30–27, 30–27)",3,5:00,NaN,main event,UFC Fight Night: MacDonald vs. Saffiedine,5f12a76ed2214b2a88d6e157c42751360d2a47dd2272dc...
2,Lightweight,Chad Laprise,Yosdenis Cedeno,"Decision (unanimous) (30–27, 30–27, 30–27)",3,5:00,NaN,main event,UFC Fight Night: MacDonald vs. Saffiedine,5f12a76ed2214b2a88d6e157c42751360d2a47dd2272dc...
3,Middleweight,Elias Theodorou,Bruno Santos,"Decision (unanimous) (29–28, 29–28, 29–28)",3,5:00,NaN,main event,UFC Fight Night: MacDonald vs. Saffiedine,5f12a76ed2214b2a88d6e157c42751360d2a47dd2272dc...
4,Welterweight,Nordine Taleb,Li Jingliang,"Decision (split) (28–29, 29–28, 30–27)",3,5:00,NaN,main event,UFC Fight Night: MacDonald vs. Saffiedine,5f12a76ed2214b2a88d6e157c42751360d2a47dd2272dc...
...,...,...,...,...,...,...,...,...,...,...
97,Middleweight,Jeremy Horn,Chuck Liddell,Technical Submission (inverted arm-triangle ch...,NaN,12:00,NaN,main event,UFC 19: Ultimate Young Guns,7489dfebb61d0b1bf9582fb6d0c2271e20bfc414557572...
98,Heavyweight,Kevin Randleman,Maurice Smith,Decision (unanimous),NaN,15:00,NaN,main event,UFC 19: Ultimate Young Guns,7489dfebb61d0b1bf9582fb6d0c2271e20bfc414557572...
99,Middleweight,Evan Tanner,Valeri Ignatov,TKO (punches and elbows),NaN,2:57,NaN,main event,UFC 19: Ultimate Young Guns,7489dfebb61d0b1bf9582fb6d0c2271e20bfc414557572...
100,Heavyweight,Pete Williams,Jason Godsey,Submission (kneebar),NaN,1:54,NaN,main event,UFC 19: Ultimate Young Guns,7489dfebb61d0b1bf9582fb6d0c2271e20bfc414557572...
